In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "NT_Site_PredNTS_Classification_ML_KmerRFE"
outPath = "Results"
foldName = "folds.pickle"

shuffle = True
seed = None

input_data_folder = "PredNTS_MathFeature_ENC"

monitor = 'val_loss'

sub_feature_count = 500

In [2]:
train_data_filename = 'Training-datasets-PredNTS_kmer.csv'
indpe_data_filename = 'independent-dataset-PredNTS_kmer.csv'

In [3]:
import os 
import pickle
import numpy as np
import pandas as pd

# import tensorflow as tf

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score, classification_report, matthews_corrcoef

from sklearn.feature_selection import RFE

import math

In [4]:
##################################################################################
##### Build k-fold functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

In [5]:
##################################################################################
##### define evaluator functions
##################################################################################

def pred2label(y_pred):
    y_pred = np.round(y_pred)
    return y_pred

In [6]:
def get_model(cw = None):
    
    model = RandomForestClassifier(n_estimators=100,
                                   criterion='gini', 
                                   class_weight=cw,
                                   bootstrap=True,
                                   oob_score=True, 
                                  )
    
    return model

# Train data preparation

In [7]:
##################################################################################
##### Read CSV data
##################################################################################

train_data_filepath = os.path.join(input_data_folder, train_data_filename)
train_data = pd.read_csv(train_data_filepath, sep=',', header=0)
train_data = train_data.drop('label', axis=1)

train_data['label'] = pd.Series([int(val.split('_')[-2])
                                 for val in train_data['nameseq']])

train_data = train_data.drop('nameseq', axis=1)

##################################################################################
##### Extract features and labels, create folds
##################################################################################

train_features = np.array(train_data.drop('label', axis=1))
train_labels = np.array(train_data['label'])
# train_labels = train_labels.reshape((train_labels.shape[0], 1))

In [8]:
##################################################################################
##### Recursive feature selection
##################################################################################

# model = DecisionTreeClassifier(criterion="gini")
model = get_model()

selector = RFE(model, n_features_to_select=sub_feature_count, step=100)
selector = selector.fit(train_features, train_labels)

feature_indices = np.where(selector.ranking_ == 1)[0]

In [9]:
##################################################################################
##### Extract features and labels, create folds
##################################################################################

train_features = train_features[:, feature_indices]

folds = build_kfold(train_features, train_labels, k=n_fold, shuffle=shuffle, seed=seed)

input_vec_shape = train_features[0].shape

## Write the k-fold dataset to file
foldPath = os.path.join(outPath, expName, "{}fold".format(n_fold))
if(not os.path.isdir(foldPath)):
    os.makedirs(foldPath)
pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

# Independent data

In [10]:
##################################################################################
##### Read CSV data
##################################################################################

indpe_data_filepath = os.path.join(input_data_folder, indpe_data_filename)
indpe_data = pd.read_csv(indpe_data_filepath, sep=',', header=0)
indpe_data = indpe_data.drop('label', axis=1)

indpe_data['label'] = pd.Series([int(val.split('_')[-2])
                                 for val in indpe_data['nameseq']])

indpe_data = indpe_data.drop('nameseq', axis=1)

##################################################################################
##### Extract features and labels, create folds
##################################################################################

indpe_features = np.array(indpe_data.drop('label', axis=1))
indpe_features = indpe_features[:, feature_indices]

indpe_labels = np.array(indpe_data['label'])
# indpe_labels = indpe_labels.reshape((indpe_labels.shape[0], 1))

# Training

In [11]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Train/Test model on all folds, generate evaluations
##################################################################################

## Create and set directory to save model
modelPath = os.path.join(outPath, expName, "{}fold".format(n_fold), "models")
if(not os.path.isdir(modelPath)):
    os.makedirs(modelPath)

i = -1
for fold in folds:
    i += 1
    
    print("\nTrain/Test model on Fold #"+str(i)+".")
    
    # adding random shuffling of the dataset for training purpose
    index_arr = np.arange(fold["X_train"].shape[0])
    index_arr = np.random.permutation(index_arr)
    
    model = get_model()
    
    ## Define the model callbacks for early stopping and saving the model. Then train model
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    
    model.fit(fold["X_train"][index_arr], fold["y_train"][index_arr].reshape(fold["y_train"].shape[0]))
    
    model_file_obj = open(current_model_path, 'wb')
    pickle.dump(model, model_file_obj)
    model_file_obj.close()
    
    ##################################################################################
    ##### Prediction and metrics for TRAIN dataset
    ##################################################################################

    y_pred = model.predict(fold["X_train"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_train"], label_pred)
    prec = precision_score(fold["y_train"],label_pred)
    mcc = matthews_corrcoef(fold["y_train"], label_pred)

    conf = confusion_matrix(fold["y_train"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
    auc = roc_auc_score(fold["y_train"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Train")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    ##################################################################################
    ##### Prediction and metrics for TEST dataset
    ##################################################################################

    y_pred = model.predict(fold["X_test"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_test"], label_pred)
    prec = precision_score(fold["y_test"],label_pred)
    mcc = matthews_corrcoef(fold["y_test"], label_pred)

    conf = confusion_matrix(fold["y_test"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
    auc = roc_auc_score(fold["y_test"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Test")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)


Train/Test model on Fold #0.

Train/Test model on Fold #1.

Train/Test model on Fold #2.

Train/Test model on Fold #3.

Train/Test model on Fold #4.


## k-fold Training evaluation

In [12]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Test,0.795546,0.805331,0.795529,0.781713,0.809346,0.592119
Train,0.996642,0.998526,0.996642,0.994753,0.998531,0.993292


In [13]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Train,0.997900,0.997899,"[0.0, 0.9978991596638656, 1.0]","[0.0, 0.002098635886673662, 1.0]","[2, 1, 0]",0.997900,0.997899,0.997901,0.995801
1,0,Test,0.779874,0.791304,"[0.0, 0.7615062761506276, 1.0]","[0.0, 0.20168067226890757, 1.0]","[2, 1, 0]",0.779913,0.761506,0.798319,0.560180
2,1,Train,0.996325,0.998945,"[0.0, 0.993704092339979, 1.0]","[0.0, 0.0010504201680672268, 1.0]","[2, 1, 0]",0.996327,0.993704,0.998950,0.992665
3,1,Test,0.819706,0.861905,"[0.0, 0.7605042016806722, 1.0]","[0.0, 0.12133891213389121, 1.0]","[2, 1, 0]",0.819583,0.760504,0.878661,0.643777
4,2,Train,0.995803,0.997893,"[0.0, 0.993704092339979, 1.0]","[0.0, 0.002098635886673662, 1.0]","[2, 1, 0]",0.995803,0.993704,0.997901,0.991614
5,2,Test,0.781513,0.786325,"[0.0, 0.773109243697479, 1.0]","[0.0, 0.21008403361344538, 1.0]","[2, 1, 0]",0.781513,0.773109,0.789916,0.563105
6,3,Train,0.996327,0.998945,"[0.0, 0.993704092339979, 1.0]","[0.0, 0.001049317943336831, 1.0]","[2, 1, 0]",0.996327,0.993704,0.998951,0.992668
7,3,Test,0.808824,0.807531,"[0.0, 0.8109243697478992, 1.0]","[0.0, 0.19327731092436976, 1.0]","[2, 1, 0]",0.808824,0.810924,0.806723,0.617653
8,4,Train,0.996852,0.998946,"[0.0, 0.9947534102833159, 1.0]","[0.0, 0.001049317943336831, 1.0]","[2, 1, 0]",0.996852,0.994753,0.998951,0.993713
9,4,Test,0.787815,0.779592,"[0.0, 0.8025210084033614, 1.0]","[0.0, 0.226890756302521, 1.0]","[2, 1, 0]",0.787815,0.802521,0.773109,0.575879


## Using k-fold Models

### Performance of each k-fold model

In [14]:
## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model_file_obj = open(current_model_path, 'rb')
    model = pickle.load(model_file_obj)
    model_file_obj.close()

    y_pred = model.predict(indpe_features)
    label_pred = pred2label(y_pred)

    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(indpe_labels, label_pred)
    prec = precision_score(indpe_labels,label_pred)
    mcc = matthews_corrcoef(indpe_labels, label_pred)

    conf = confusion_matrix(indpe_labels, label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
    auc = roc_auc_score(indpe_labels, y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Independent")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.641306,0.262287,0.64173,0.642365,0.641096,0.214664


In [15]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Independent,0.635918,0.258449,"[0.0, 0.6403940886699507, 1.0]","[0.0, 0.3649706457925636, 1.0]","[2, 1, 0]",0.637712,0.640394,0.635029,0.208172
1,1,Independent,0.643265,0.259259,"[0.0, 0.6206896551724138, 1.0]","[0.0, 0.3522504892367906, 1.0]","[2, 1, 0]",0.634220,0.620690,0.647750,0.204023
2,2,Independent,0.635102,0.262646,"[0.0, 0.6650246305418719, 1.0]","[0.0, 0.37084148727984345, 1.0]","[2, 1, 0]",0.647092,0.665025,0.629159,0.221654
3,3,Independent,0.641633,0.264000,"[0.0, 0.6502463054187192, 1.0]","[0.0, 0.36007827788649704, 1.0]","[2, 1, 0]",0.645084,0.650246,0.639922,0.219517
4,4,Independent,0.650612,0.267081,"[0.0, 0.6354679802955665, 1.0]","[0.0, 0.34637964774951074, 1.0]","[2, 1, 0]",0.644544,0.635468,0.653620,0.219952


### Mean score with k-fold models

In [16]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model_file_obj = open(current_model_path, 'rb')
    model = pickle.load(model_file_obj)
    model_file_obj.close()

    y_pred = model.predict(indpe_features)
    total_pred += y_pred
    all_preds.append(y_pred)
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.661224,0.276371,0.671698,0.64532,0.664384,0.236434


### Voting score with k-fold models

In [17]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model_file_obj = open(current_model_path, 'rb')
    model = pickle.load(model_file_obj)
    model_file_obj.close()

    y_pred = model.predict(indpe_features)
    vote_pred = pred2label(y_pred)
    total_pred += vote_pred
    all_preds.append(vote_pred)
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.661224,0.276371,0.671698,0.64532,0.664384,0.236434


## Using New Model

Train model on full data from training. Predict and evaluate on Independent data.

In [18]:
model = get_model(cw={0:1, 1:0.1})
    
## Define the model callbacks for early stopping and saving the model. Then train model
current_model_path = os.path.join(modelPath, "_fullModel.hdf5")

# adding random shuffling of the dataset for training purpose
index_arr = np.arange(train_features.shape[0])
index_arr = np.random.permutation(index_arr)

model.fit(train_features[index_arr], train_labels[index_arr].reshape(train_labels.shape[0]))

RandomForestClassifier(n_estimators=300, oob_score=True)

In [19]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

y_pred = model.predict(indpe_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
auc = roc_auc_score(indpe_labels, y_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.644898,0.264228,0.643093,0.640394,0.645793,0.217064


In [20]:
print(classification_report(indpe_labels, np.round(y_pred).astype(int)))

              precision    recall  f1-score   support

           0       0.90      0.65      0.75      1022
           1       0.26      0.64      0.37       203

    accuracy                           0.64      1225
   macro avg       0.58      0.64      0.56      1225
weighted avg       0.79      0.64      0.69      1225

